In [ ]:
import pandas as pd
import urllib.request
import tarfile
import os
import zipfile
import video_utils as vu


In [ ]:

download_url = "https://download.microsoft.com/download/3/c/a/3ca92c78-1c4a-4a91-a7ee-6980c1d242ec/MS-ASL.zip"
dataset_path = "msasl_dataset"
destination_path = "msasl.zip"

# MediaPipe hand landmark connections for 2D plotting
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),  # Index
    (5, 9), (9, 10), (10, 11), (11, 12),  # Middle
    (9, 13), (13, 14), (14, 15), (15, 16),  # Ring
    (13, 17), (17, 18), (18, 19), (19, 20),  # Pinky
    (0, 17)  # Palm base connection
]

In [ ]:
urllib.request.urlretrieve(download_url, destination_path)

# Extract the ZIP file
with zipfile.ZipFile(destination_path, 'r') as zip_ref:
    zip_ref.extractall("msasl_dataset")

In [ ]:
MSASL_test = pd.read_json("{}/MS-ASL/MSASL_test.json".format(dataset_path))

In [ ]:
# Example Usage:
import media_pipe_handler as mph
#mph.extract_hand_keypoints("clips/medicine_UxuLj17UyDU.mkv", "output_keypoints/")

mph.normalize_keypoints("output_keypoints/medicine_UxuLj17UyDU.npy", "output_keypoints/normalized/medicine_UxuLj17UyDU.npy")

Normalized keypoints saved to: output_keypoints/normalized/medicine_UxuLj17UyDU.npy


In [6]:
def visualize_hand_keypoints_2d(npy_file):  # TODO: Fix this, only works inside a jupyter notebook, doesn't work in a script
    keypoints = np.load(npy_file)  # Shape: (num_frames, 126)
    
    if keypoints.shape[1] != 126:
        print("Error: Expected shape (num_frames, 126) but got", keypoints.shape)
        return

    num_frames = keypoints.shape[0]
    keypoints = keypoints.reshape(num_frames, 2, 21, 3)  # (frames, hands, landmarks, x/y/z)

    fig, ax = plt.subplots()
    ax.set_xlim(0, 1)  # Normalized coordinates (0 to 1)
    ax.set_ylim(0, 1)
    ax.invert_yaxis()  # Invert Y-axis to match image coordinates

    # Initialize hand dots and connection lines
    left_hand_dots, = ax.plot([], [], 'ro', markersize=5)  # Red for left hand
    right_hand_dots, = ax.plot([], [], 'bo', markersize=5)  # Blue for right hand
    left_hand_lines = [ax.plot([], [], 'r-')[0] for _ in HAND_CONNECTIONS]
    right_hand_lines = [ax.plot([], [], 'b-')[0] for _ in HAND_CONNECTIONS]

    def update(frame):
        ax.set_title(f"Frame: {frame}/{num_frames}")

        left_hand, right_hand = keypoints[frame, :, :, :2]  # Extract only X, Y

        # Update scatter points
        left_hand_dots.set_data(left_hand[:, 0], left_hand[:, 1])
        right_hand_dots.set_data(right_hand[:, 0], right_hand[:, 1])

        # Update lines
        for i, (start, end) in enumerate(HAND_CONNECTIONS):
            left_hand_lines[i].set_data([left_hand[start, 0], left_hand[end, 0]],
                                        [left_hand[start, 1], left_hand[end, 1]])
            right_hand_lines[i].set_data([right_hand[start, 0], right_hand[end, 0]],
                                         [right_hand[start, 1], right_hand[end, 1]])

        return left_hand_dots, right_hand_dots, *left_hand_lines, *right_hand_lines

    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=50, blit=False)
    
    return HTML(ani.to_jshtml())  # Display animation inside Jupyter

visualize_hand_keypoints_2d("output_keypoints/medicine_UxuLj17UyDU.npy")

<IPython.core.display.Javascript object>